## RFM Analizi 2009-2010 vs. 2010-2011

Bu çalışmada Online Retail II veri seti kullanılarak [2009-2010](https://www.kaggle.com/cemalcici/rfm-analizi-ile-m-teri-segmentasyonu-2009-2010) ve [2010-2011](https://www.kaggle.com/cemalcici/rfm-analizi-ile-m-teri-segmentasyonu-2010-2011) dönemlerinin karşılaştırılması yapılmıştır. Proje hakkında bilgilere dönemlere bağlı linklere tıklayarak ulaşabilirsiniz.

In [ ]:
import pandas as pd # Veri Manipülasyonu
import numpy as np # Matris İşlemleri
import datetime as dt # Tarih Saat İşlemleri
import plotly.express as px # Görselleştirme İşlemleri
import warnings # Uyarılar


warnings.filterwarnings("ignore") # Uyarıları görmezden geldik.
pd.set_option('display.expand_frame_repr', False)

In [ ]:
def load_retail_data(pathname):
    """
    Online Retail II veri setlerini almak için oluşturulan fonksiyon.
    
    Parameters
    -----------
    pathname str, optional
        Veri setinin dizini.
    
    Returns
    -----------
    pd.DataFrame
        Dizine göre dönen veri seti.
    """
    return pd.read_csv(pathname, sep=";")

In [ ]:
def data_prep_for_rfm(dataframe):
    """
    RFM için Online Retail özelinde veri ön işleme işlemlerinin toplandığı fonksiyon
    
    Parameters
    ------------
    dataframe pd.DataFrame
        RFM analizi için veri seti
    
    Returns
    ------------
    dataframe pd.DataFrame
        RFM için hazırlanmış veri seti
    
    """
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~(dataframe["Invoice"].str.contains("C"))]
    dataframe["Customer ID"] = dataframe["Customer ID"].astype(int).astype(str)
    dataframe["InvoiceDate"] = pd.to_datetime(dataframe["InvoiceDate"])
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    return dataframe

In [ ]:
def create_rfm_table(dataframe:pd.DataFrame, dataframe_id:str, rfm_grid:dict, segment_list=False):
    """
    RFM Analizi için tablonun oluşturulmasını sağlayan fonksiyon.
    Bu fonksiyon işlemleri 4 adımda tamamlamaktadır.
        1. Adım: RFM metriklerinin oluşturulması
            RFM metriklerini oluştururken gruplanacak id değerini ve R-F-M değerlerini kullanıcı kendi belirler.
        2. Adım: RFM skorlarının oluşturulması.
            RFM skorları oluşturulurken metrik tablosundan yararlanılır. Değerler 0-20, 21-40, 41-60, 61-80 ve 81-100 yüzdeliklerine göre 5 parçaya bölünmüştür. 
            Frequency'de iki yüzdelik aralığı arasında kalan değerlerin ilkine etiket atalamak adına rank() metodu kullanılmıştır.
        3. Adım: Segmentlerin oluşturulması.
            R ve F skorlarına göre RegEx yapısı kullanılarak segmentler oluşturulmuştur.
            Referans alınan kaynak: https://guillaume-martin.github.io/rfm-segmentation-with-python.html
    
    Parameters
    -----------
    dataframe pd.DataFrame
        RFM tablosunun oluşması için gereken veri yapısı
    dataframe_id str
        Probleme bağlı olarak ele alınacak id sütunu
    rfm_grid dict
        Probleme bağlı olara kullanılacak toplulaştırma sözlüğü
    
    Returns
    -----------
    rfm_table pd.DataFrame
        Oluşturulan RFM tablosu.
    seg_map.values() list
        Segment listesi. 
    
    Examples
    -----------
    >>> ...
    >>> analyse_date = dt.datetime(2011, 12, 11)
    >>> agg_dict = {"InvoiceDate": lambda date: (today_date - date.max()).days,
                    "Invoice": "nunique",
                    "TotalPrice": "sum"}
    >>> rfm = create_rfm_table(df, "Customer ID", agg_dict)
    >>> rfm.head()
         Customer ID  Recency  Frequency  Monetary recency_score frequency_score monetary_score RFM_SCORE       segment
    0       12346      326          1  77183.60             1               1              5        11      hibernating
    1       12347       40          7   4310.00             4               5              5        45  loyal_customers
    2       12348       76          4   1797.24             3               4              4        34  loyal_customers
    3       12349       19          1   1757.55             4               1              4        41        promising
    4       12350      311          1    334.40             1               1              2        11      hibernating
    """
    # Adım 1: RFM Metriklerinin oluşturulması
    rfm_table = dataframe.groupby(dataframe_id).agg(rfm_grid)
    rfm_table.columns = ["Recency", "Frequency", "Monetary"]
    rfm_table = rfm_table[rfm_table["Monetary"] > 0]
    rfm_table.reset_index(inplace=True)
    
    # Adım 2: RFM Skorlarının oluşturulması
    rfm_table["recency_score"] = pd.qcut(rfm_table['Recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm_table["frequency_score"] = pd.qcut(rfm_table['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm_table["monetary_score"] = pd.qcut(rfm_table['Monetary'], 5, labels=[1, 2, 3, 4, 5])
    
    # Adım 3: Segmentlerin oluşturulması
    rfm_table["RFM_SCORE"] = (rfm_table['recency_score'].astype(str) + rfm_table['frequency_score'].astype(str))
    seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
    }
    rfm_table['segment'] = rfm_table['RFM_SCORE'].replace(seg_map, regex=True)
    
    
    # Adım 4: RFM tablosunun döndürülmesi
    if segment_list:        
        return rfm_table, seg_map.values() 
    return rfm_table

In [ ]:
online_retail_2009_2010_df = load_retail_data("../input/online-retail-ii-uci-two-peroid/online_retail_II_2009_2010.csv")
online_retail_2010_2011_df = load_retail_data("../input/online-retail-ii-uci-two-peroid/online_retail_II_2010_2011.csv")

In [ ]:
online_retail_2009_2010_df = data_prep_for_rfm(online_retail_2009_2010_df)
online_retail_2010_2011_df = data_prep_for_rfm(online_retail_2010_2011_df)

In [ ]:
analyse_date_2010 = dt.datetime(2010, 12, 13)
agg_dict_2010 = {"InvoiceDate": lambda date: (analyse_date_2010 - date.max()).days,
                 "Invoice": "nunique",
                 "TotalPrice": "sum"}
rfm_2009_2010 = create_rfm_table(online_retail_2009_2010_df, "Customer ID", agg_dict_2010)
rfm_2009_2010["period"] = "2009_2010"

In [ ]:
analyse_date_2011 = dt.datetime(2011, 12, 11)
agg_dict_2011 = {"InvoiceDate": lambda date: (analyse_date_2011 - date.max()).days,
                 "Invoice": "nunique",
                 "TotalPrice": "sum"}
rfm_2010_2011 = create_rfm_table(online_retail_2010_2011_df, "Customer ID", agg_dict_2011)
rfm_2010_2011["period"] = "2010_2011"

In [ ]:
temp_df = pd.concat([rfm_2009_2010[["segment", "period"]], rfm_2010_2011[["segment", "period"]]], ignore_index=True)
fig = px.treemap(temp_df, path=["segment", "period"], title="RFM Segments 2009-2010 vs. 2010-2011")
fig.show()